# Assistants API - Knowledge Retrieval 

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval

https://community.openai.com/t/new-assistants-api-a-potential-replacement-for-low-level-rag-style-content-generation/475677 

Watch:

https://youtu.be/5rcjGjgJNQc?t=600&si=d9OtX0nMi2Rv0fQV 

References:

https://community.openai.com/t/assistants-api-retrieval-pricing-how-much-does-this-cost/485188/8

https://medium.com/madhukarkumar/what-does-openais-announcement-mean-for-retrieval-augmented-generation-rag-and-vector-only-54bfc34cba2c

https://www.youtube.com/watch?v=ClfyQNkTeUc

https://www.pinecone.io/learn/assistants-api-canopy/




![Alt text](ret.jpg "Assistants")

![Alt text](objects.jpeg "Assistants_Objects")

https://cobusgreyling.medium.com/openai-assistant-with-retriever-tool-08e9158ca900 

In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv("E:/Python/.env") # read local .env file

In [2]:
client : OpenAI = OpenAI()

### Knowledge Retrieval

Retrieval augments the Assistant with knowledge from outside its model, such as proprietary product information or documents provided by your users. Once a file is uploaded and passed to the Assistant, OpenAI will automatically chunk your documents, index and store the embeddings, and implement vector search to retrieve relevant content to answer user queries.

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval 



### How it works

The model then decides when to retrieve content based on the user Messages. The Assistants API automatically chooses between two retrieval techniques:

1. it either passes the file content in the prompt for short documents, or
2. performs a vector search for longer documents

Retrieval currently optimizes for quality by adding all relevant content to the context of model calls. We plan to introduce other retrieval strategies to enable developers to choose a different tradeoff between retrieval quality and model usage cost.

https://platform.openai.com/docs/assistants/tools/how-it-works


### Step 1a: Upload the file for an Assistant

In [3]:
file = client.files.create(
    file = open("zia_profile.pdf", "rb"),
    purpose= "assistants"
)
print(file)

FileObject(id='file-7qDEgiBxzB8NJXmo3XkfIZO8', bytes=48802, created_at=1702595820, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


### Step 1b: Create an Assistant

In [5]:
from openai.types.beta.assistant import Assistant

assistant: Assistant = client.beta.assistants.create(
    name="Student Support Assistant",
    instructions="You are a student support chatbot. Use your knowledge based to best respond to quires about Zia U.Khan",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
    )

In [9]:
print(assistant)

Assistant(id='asst_wL6o5K2QeCbJlSaFU9GCd52N', created_at=1702596376, description=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='You are a student support chatbot. Use your knowledge based to best respond to quires about Zia U.Khan', metadata={}, model='gpt-3.5-turbo-1106', name='Student Support Assistant', object='assistant', tools=[ToolRetrieval(type='retrieval')])


### Step 2: Create a Thread

In [6]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()
print(thread)

Thread(id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', created_at=1702596580, metadata={}, object='thread')


### Step 3: Add a Message to a Thread

In [7]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= "When and Which city zia u khan was born?"
)


In [10]:
print(message)

ThreadMessage(id='msg_LM8M8pAS8DRe460QH3w7WmhF', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='When and Which city zia u khan was born?'), type='text')], created_at=1702596859, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK')


### Step 4: Run the Assistant

In [14]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions= "Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan"
)
print(run)


Run(id='run_6z00KQvMj8Z8eMUuPflBpS3t', assistant_id='asst_wL6o5K2QeCbJlSaFU9GCd52N', cancelled_at=None, completed_at=None, created_at=1702597629, expires_at=1702598229, failed_at=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


### Step 5: Check the Run status

In [24]:
import time

def print_run_and_check_status(run):
    current_status = run.status
    while current_status != "completed":
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        current_status = run.status
        print(current_status)
        if run.status == "failed":
            break

    if run.status == "completed":
        print(run)
        
    

In [16]:
print_run_and_check_status(run)

completed
Run(id='run_6z00KQvMj8Z8eMUuPflBpS3t', assistant_id='asst_wL6o5K2QeCbJlSaFU9GCd52N', cancelled_at=None, completed_at=1702597631, created_at=1702597629, expires_at=None, failed_at=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702597629, status='completed', thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


### Step 6: Display the Assistant's Response

In [17]:
from typing import List

messages: List[ThreadMessage] = client.beta.threads.messages.list(thread_id= thread.id)




In [18]:
type(messages)


openai.pagination.SyncCursorPage[ThreadMessage]

In [19]:
for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)

user: When and Which city zia u khan was born?
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961. His family, including his father who was a late Lt. Col. in the army, moved throughout Pakistan during his childhood【7†source】.
assistant: Zia U. Khan was born in Sialkot in 1961【6†source】.


In [21]:
# Add another message to the thread
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= "What do you know about his education?"
)

# Run the Assistant
run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions= "Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan"
)

# Check the Assistant's Status
print_run_and_check_status(run)

#  Updating Thread messages on messages(variable)
messages: List[ThreadMessage] = client.beta.threads.messages.list(thread_id= thread.id)

# Display the Assistant's Response
for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

in_progress
completed
Run(id='run_Ts9S1EZSn30QGerdCPvRtxoD', assistant_id='asst_wL6o5K2QeCbJlSaFU9GCd52N', cancelled_at=None, completed_at=1702598828, created_at=1702598826, expires_at=None, failed_at=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702598826, status='completed', thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])
user: When and Which city zia u khan was born?
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961. His family, including his father who was a late Lt. Col. in the army, moved throughout Pakistan during his childhood【7†source】.
assistant: Zia U. Khan was born in Sialkot in 1961【6†source】.
user: How much he is qualified?
user: What do you know about his 

In [22]:
# Add another message to the thread
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= "what is his teaching strategy?"
)

# Run the Assistant
run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions= "Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan"
)

# Check the Assistant's Status
print_run_and_check_status(run)

#  Updating Thread messages on messages(variable)
messages: List[ThreadMessage] = client.beta.threads.messages.list(thread_id= thread.id)

# Display the Assistant's Response
for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

in_progress
in_progress
in_progress
completed
Run(id='run_YGii8sNnofhRQz2vbVgAtoAp', assistant_id='asst_wL6o5K2QeCbJlSaFU9GCd52N', cancelled_at=None, completed_at=1702598976, created_at=1702598971, expires_at=None, failed_at=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702598972, status='completed', thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])
user: When and Which city zia u khan was born?
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961. His family, including his father who was a late Lt. Col. in the army, moved throughout Pakistan during his childhood【7†source】.
assistant: Zia U. Khan was born in Sialkot in 1961【6†source】.
user: How much he is qualified?
user: Wha

In [25]:
# Add another message to the thread
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= "who support him?"
)

# Run the Assistant
run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions= "Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan"
)

# Check the Assistant's Status
print_run_and_check_status(run)

#  Updating Thread messages on messages(variable)
messages: List[ThreadMessage] = client.beta.threads.messages.list(thread_id= thread.id)

# Display the Assistant's Response
for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

in_progress
completed
Run(id='run_jIfT4oSJdkV9kcYHemt0drh8', assistant_id='asst_wL6o5K2QeCbJlSaFU9GCd52N', cancelled_at=None, completed_at=1702599253, created_at=1702599251, expires_at=None, failed_at=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702599251, status='completed', thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])
user: When and Which city zia u khan was born?
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961. His family, including his father who was a late Lt. Col. in the army, moved throughout Pakistan during his childhood【7†source】.
assistant: Zia U. Khan was born in Sialkot in 1961【6†source】.
user: How much he is qualified?
user: What do you know about his 

In [26]:
# Add another message to the thread
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= "What do you know about his educational initiatives?"
)

# Run the Assistant
run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions= "Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan"
)

# Check the Assistant's Status
print_run_and_check_status(run)

#  Updating Thread messages on messages(variable)
messages: List[ThreadMessage] = client.beta.threads.messages.list(thread_id= thread.id)

# Display the Assistant's Response
for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

in_progress
in_progress
completed
Run(id='run_w5EwwR29gk1ekDmG0ikHS0XH', assistant_id='asst_wL6o5K2QeCbJlSaFU9GCd52N', cancelled_at=None, completed_at=1702599404, created_at=1702599401, expires_at=None, failed_at=None, file_ids=['file-7qDEgiBxzB8NJXmo3XkfIZO8'], instructions='Please address the student as Pakistani, He is a newly enrolled student of PIAIC and want to know about Zia U Khan', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702599401, status='completed', thread_id='thread_OXAjHGNiNOCnlbeqtCgoE1WK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])
user: When and Which city zia u khan was born?
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961. His family, including his father who was a late Lt. Col. in the army, moved throughout Pakistan during his childhood【7†source】.
assistant: Zia U. Khan was born in Sialkot in 1961【6†source】.
user: How much he is qualified?
user: What do you kno

In [31]:
for m in reversed(messages.data):
    role = m.role
    content = m.content[0].text.value
    
    if role == "user":
        print("\033[91m" + role + ": " + content + "\033[0m")  # Red color (role1)
    elif role == "assistant":
        print("\033[92m" + role + ": " + content + "\033[0m")  # Green color (role2)
    else:
        print(role + ": " + content)  # Default color



user: When and Which city zia u khan was born?
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961. His family, including his father who was a late Lt. Col. in the army, moved throughout Pakistan during his childhood【7†source】.
assistant: Zia U. Khan was born in Sialkot in 1961【6†source】.
user: How much he is qualified?
user: What do you know about his education?
assistant: Zia U. Khan completed his Master's in Economics from Karachi in 1987. Subsequently, he pursued triple master's degrees at Arizona State University (ASU) - an MBA, MS, and MAC. Additionally, he obtained certifications as a Certified Public Accountant (CPA) and a Certified Management Accountant (CMA) in the USA, completing all these qualifications by December 1992【13†source】.
user: what is his teaching strategy?
assistant: I couldn't find specific information about Zia U. Khan's teaching strategy in the provided document. However, it is evident that he has been involved in teaching and education ini